In [1]:
import pandas as pd
import numpy as np
import datetime
import os
os.getcwd()

'/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/20190118_Zips_3_DMA_and_States'

In [2]:
store_list=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/StoreList_20190107.xlsx",dtype=str)
store_list=store_list[~pd.isnull(store_list['storenumber'])]
closed_stores=store_list[store_list['status']=="Closed"]
store_list=store_list[store_list['status']!="Closed"]
store_list=store_list[store_list['storenumber']!="0930"]
closed_stores=closed_stores.append(store_list['storenumber']=="0930")
store_list=store_list[store_list['storenumber']!="nan"].reset_index()
del store_list['index']

store_list['Latitude']=store_list['Latitude'].astype(float)
store_list['Longitude']=store_list['Longitude'].astype(float)
store_list['zip']=store_list['zip'].apply(lambda x: x.split("-")[0])

# Manually update the zip for the 11 stores
store_zip_dic_updated={}
store_zip_dic_updated.update({"1333":"36695"})
store_zip_dic_updated.update({"1349":"07747"})
store_zip_dic_updated.update({"1436":"78411"})

store_zip_dic_updated.update({"1245":"68133"})
store_zip_dic_updated.update({"1152":"70810"})
store_zip_dic_updated.update({"1282":"70803"})

store_zip_dic_updated.update({"1386":"62711"})
store_zip_dic_updated.update({"1467":"63146"})
store_zip_dic_updated.update({"1252":"73013"})

store_zip_dic_updated.update({"1397":"28054"})
store_zip_dic_updated.update({"1653":"46074"})

store_zip_dic_updated.update({"1390":"39440"}) # Changed
store_zip_dic_updated.update({"1464":"07901"}) # Changed

store_zip_dic_updated.update({"0989":"70130"}) # Changed
store_zip_dic_updated.update({"1457":"08873"}) # Changed
store_zip_dic_updated.update({"1432":"30281"}) # Changed

store_without_zips=store_list[store_list['storenumber'].isin(store_zip_dic_updated.keys())]
store_with_zips=store_list[~store_list['storenumber'].isin(store_zip_dic_updated.keys())]
store_without_zips['zip']=store_without_zips['storenumber'].apply(lambda x: store_zip_dic_updated[x])

store_list=store_with_zips.append(store_without_zips).reset_index()
store_list=store_list.sort_values(['index'])
del store_list['index']
store_list=store_list.reset_index()
del store_list['index']

store_zip5_dic=store_list.set_index(['storenumber'])['zip'].to_dict()
store_list=store_list.rename(columns={"zip":"store_zip"})

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:4528: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  other.index).difference(self.columns).tolist()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
DMA_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,usecols=[0,2],dtype=str).drop_duplicates()
DMA_zips.columns=['zip_cd','DMA']

Dallas_zips=DMA_zips[DMA_zips['DMA']=="DALLAS-FT. WORTH"]
NewOrleans_zips=DMA_zips[DMA_zips['DMA']=="NEW ORLEANS"]
BatonRouge_zips=DMA_zips[DMA_zips['DMA']=="BATON ROUGE"]

In [4]:
stores_in_Dallas=store_list[store_list['store_zip'].isin(Dallas_zips['zip_cd'].tolist())]
stores_in_NewOrleans=store_list[store_list['store_zip'].isin(NewOrleans_zips['zip_cd'].tolist())]
stores_in_BatonRouge=store_list[store_list['store_zip'].isin(BatonRouge_zips['zip_cd'].tolist())]

In [5]:
stores_in_BatonRouge.head(2)

,storenumber,Address,city,state,store_zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate
281,1022,1910 SW Rail Road Ave,Hammond,LA,70403,2010-04-23 00:00:00,30.483038,-90.457365,Franchised,Jason Finley,108.210.88.87,Y,Same,nan
394,1243,906 W Oak Street,Amite,LA,70422,2014-11-26 00:00:00,30.726929,-90.522259,Franchised,Matt Montgomery,97.80.24.205,Y,Same,nan


In [6]:
TA_zips_by_store=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/SmoothieKing_TA_of_3_miles_zips_JL_2019-01-09.xlsx",sheetname="zip_TA",dtype=str)
TA_zips_by_store=TA_zips_by_store.rename(columns={"store":"storenumber","zip":"TA_zip"})

zips_for_Dallas=pd.merge(stores_in_Dallas,TA_zips_by_store,on="storenumber",how="left")
zips_for_Dallas['Store_DMA']="DALLAS-FT. WORTH"
zips_for_Dallas_unique_zip=zips_for_Dallas[['Store_DMA','TA_zip']].drop_duplicates()

zips_for_NewOrleans=pd.merge(stores_in_NewOrleans,TA_zips_by_store,on="storenumber",how="left")
zips_for_NewOrleans['Store_DMA']="NEW ORLEANS"
zips_for_NewOrleans_unique_zip=zips_for_NewOrleans[['Store_DMA','TA_zip']].drop_duplicates()

zips_for_BatonRouge=pd.merge(stores_in_BatonRouge,TA_zips_by_store,on="storenumber",how="left")
zips_for_BatonRouge['Store_DMA']="BATON ROUGE"
zips_for_BatonRouge_unique_zip=zips_for_BatonRouge[['Store_DMA','TA_zip']].drop_duplicates()


In [7]:
TA_zips=TA_zips_by_store[['TA_zip']].drop_duplicates()

state_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,usecols=[0,6,7],dtype=str).drop_duplicates()
state_zips.columns=['TA_zip','ST',"state"]
state_zips=state_zips.drop_duplicates()
states_list=['LA','MS',"TX","FL","AL","NC","SC","TN","GA","KS","IN","MO","VA","MD","DC"]
len(states_list)
state_zips=state_zips[state_zips['ST'].isin(states_list)].drop_duplicates()
state_zips=pd.DataFrame({"TA_zip":"20532","ST":"DC","state":"Dist. of Columbia"},index=[0]).append(state_zips)

In [8]:
TA_zips_for_state=pd.merge(TA_zips,state_zips,on="TA_zip",how="left").drop_duplicates()
TA_zips_for_others=TA_zips_for_state[pd.isnull(TA_zips_for_state['ST'])]
TA_zips_for_state=TA_zips_for_state[~pd.isnull(TA_zips_for_state['ST'])]


In [9]:
import zipcodes
TA_zips_for_others=TA_zips_for_others.reset_index()
TA_zips_for_others['st_check']=np.nan
for i in range(len(TA_zips_for_others)):
    try:
        TA_zips_for_others['st_check'][i]=zipcodes.matching(TA_zips_for_others['TA_zip'][i])[0]['state']
    except:
        TA_zips_for_others['st_check'][i]=np.nan

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/TA/New_TA_20190108/20190118_Zips_3_DMA_and_States/SK_zip_list_file_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")


zips_for_Dallas_unique_zip.to_excel(writer,"Dallas_unique_zip",index=False)
zips_for_NewOrleans_unique_zip.to_excel(writer,"NewOrleans_unique_zip",index=False)
zips_for_BatonRouge_unique_zip.to_excel(writer,"BatonRouge_unique_zip",index=False)
TA_zips_for_state.to_excel(writer,"zips_for_15_states_in_TA",index=False)


zips_for_Dallas.to_excel(writer,"zips_for_Dallas_by_store",index=False)
zips_for_NewOrleans.to_excel(writer,"zips_for_NewOrleans_by_store",index=False)
zips_for_BatonRouge.to_excel(writer,"zips_for_BatonRouge_by_store",index=False)
TA_zips_for_others.to_excel(writer,"zips_for_other_states_in_TA",index=False)

writer.save()
